In [1]:
import pandas as pd
import re, nltk, random, os, json
from nltk.corpus import stopwords
from rank_bm25 import BM25Okapi, BM25L, BM25Plus

In [2]:
from simpletransformers.question_answering import QuestionAnsweringModel
import json
import os
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
1234

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('distilbert', 'distilbert-base-uncased-distilled-squad', use_cuda=False, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'use_cuda': False})

In [3]:
idx = 0
to_predict = []
for context in context_sentences:
    qas = []
    for prop in cand_properties:
        qas.append({'question': cand_properties[prop], 'id': idx})
        idx += 1
    to_predict.append({'context': context, 'qas':qas})

NameError: name 'context_sentences' is not defined

In [116]:
stop_words = set(stopwords.words('english'))

regex = re.compile('[^a-zA-Z0-9.,]')
#First parameter is the replacement, second parameter is your input string
def preprocessingText(doc,enableStopword=False):
#     doc = regex.sub(' ', doc)
    if enableStopword:
        doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc.lower().replace("\n","")

df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['title'] = df_wiki['title'].str.replace("How to ", "")
for col in ['overview', 'headline', 'text', 'sectionLabel', 'title']:
    df_wiki[col] = [preprocessingText(str(i), False) for i in df_wiki[col]]
    
df_wiki['title'] = [i if not i[-1].isdigit() else i[:-1] for i in df_wiki['title']]

In [64]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/jarana/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [6]:
df_wiki_in_domains = df_wiki[df_wiki.overview.str.contains("|".join(domains))]

NameError: name 'domains' is not defined

In [517]:
# with open('data/dialog/out/bert_bert-large-uncased_defaultWithNegativeTrain.out') as json_file:
#     print(json_file)
#     data = json.load(json_file)
#     for p in data['people']:
#         print('Name: ' + p['name'])
#         print('Website: ' + p['website'])
f = open('data/dialog/out/bert_bert-large-uncased_defaultWithNegativeTrain.out', "r")
x = f.read()
x = x.replace("'", '\"') 
# x = x.replace("\", "\\") 

y = json.loads(x)

JSONDecodeError: Expecting ',' delimiter: line 1 column 29029 (char 29028)

In [508]:
a ="{'correct_text': {'7929': 'Fremont', '7929': 'Fremont'}}"
a = 

In [510]:
json.loads(a)['correct_text']

{'7929': 'Fremont'}

'{"correct_text": {"7929": "Fremont", "7929": "Fremont"}}'

In [266]:
import glob, re, math

def camel_case_split(str): 
    words = [[str[0]]] 
    for c in str[1:]: 
        if words[-1][-1].islower() and c.isupper(): 
            words.append(list(c)) 
        else: 
            words[-1].append(c) 

    return " ".join([i.lower() for i in [''.join(word) for word in words]])    

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()

schema = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/train/schema.json")
schema = pd.concat([schema, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/dev/schema.json")])
schema = pd.concat([schema, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/test/schema.json")])

tmp = []
special = {"buse": "bus", "rentalcar": "rent car", "ridesharing": "taxi", "messaging": "message"}
for i in schema.service_name.tolist():
    n = i.split("_")[0].lower()
    if n[-1] == "s":
        n = n[:-1]
    if n in special:
        n = special[n]
    
    tmp.append(n)
schema['service_name'] = tmp
domains = schema.service_name.unique().tolist()

orgType = pd.read_csv("data/dialog/all-layers-types.csv")
orgProp = pd.read_csv("data/dialog/all-layers-properties.csv")

schemaCorpus = [camel_case_split(i[0]) + " #SEP# " + cleanhtml(i[1]).lower() for i in orgType[['label', 'comment']].values]
bm25s = BM25L([doc.split() for doc in schemaCorpus])

sProp2Desc = {camel_case_split(i[0]):cleanhtml(i[1]).lower() for i in np.concatenate([orgProp[['label', 'comment']].values, orgType[['label', 'comment']].values])}
sType2Prop = {}
sType2Subtype = {}
for i,j,k in orgType[['label', 'subTypeOf', 'properties']].values:
    if type(j) == str:
        sType2Subtype[camel_case_split(i)] = [camel_case_split(z) for z in j.replace("http://schema.org/", "").split(", ")]
                           
    if type(k) == str:
        sType2Prop[camel_case_split(i)] = [camel_case_split(z) for z in k.replace("http://schema.org/", "").split(", ")]
    

In [223]:
corpus = []
doc2title = {}
for name, group in df_wiki.groupby("title"):
    doc = []
    for col in ["overview", "sectionLabel", "headline", "text"]:
#     for col in ["title", "text"]:
        doc.append(" ".join(group[col].unique().tolist()))
        
    doc = name+" #SEP# "+ " ".join(doc)
    corpus.append(doc)
    doc2title[doc] = name
# corpus = df_wiki.overview.unique().tolist()
# bm25 = BM25Plus([doc.split() for doc in corpus])
bm25w = BM25L([doc.split() for doc in corpus])
# bm25 = BM25Okapi([tokenizer.tokenize(doc) for doc in corpus])

In [456]:
# distant
from textblob import TextBlob
import nltk
unknown_services = ['alarm', 'weather']
matches = []
retrieved_properties = []
for service, desc in schema[['service_name', 'description']].values:
#   ignore weather and alarm
    if service in unknown_services:
        continue
        
    types = [i.split(" #SEP# ")[0] for i in bm25s.get_top_n(service.split(), schemaCorpus, n=5)]
    wiki2desc = dict(i.split(" #SEP# ") for i in bm25w.get_top_n(preprocessingText(desc).split(), corpus, n=5))

    #   get subtypes
    subtypes = []
    for t in types:
        if t in sType2Subtype:
            subtypes.extend(sType2Subtype[t])
    subtypes = list(set(subtypes))
    
    cand_props = []
    for t in types:
        if t in sType2Prop:
            cand_props.extend(sType2Prop[t])
            retrieved_properties.extend(sType2Prop[t])
    cand_props += types
    
    seen_prop = [""]
    for prop in cand_props:
        if prop in schema.service_name.unique().tolist():
            continue
        propLast = prop.split()[-1]
        #   check if nous
        tokenized = nltk.word_tokenize(propLast)
        nouns = [word  for (word, pos) in nltk.pos_tag(tokenized) if(pos[:2] == 'NN')]
        if len(nouns) == 0:
            propLast = ""
            
        for wiki in wiki2desc:
            for sentence in wiki2desc[wiki].split(". "):
                found_exact_match = False
                exact_match_start = sentence.find(prop)
                if exact_match_start > -1:
                    matches.append([sProp2Desc[prop], prop, exact_match_start, sentence])
                    found_exact_match = True
                    
                if propLast not in seen_prop and not found_exact_match:

                    exact_match_start = sentence.find(propLast)
                    if exact_match_start > -1:
                        matches.append([sProp2Desc[prop], propLast, exact_match_start, sentence])
        seen_prop.append(propLast)

In [458]:
len(matches)

44672

In [439]:
idx = 1
trainData = []
for i in matches:
    question, text, answer_start, context = i
    neg_question = random.choice(list(sProp2Desc.values()))
    while neg_question == question:
        neg_question = random.choice(list(sProp2Desc.values()))
                        
    tmp = {"context": context, 'qas':[{'id':str(idx+1), 'is_impossible': True, 'question': neg_question, 'answers':[]},{'id':str(idx), 'is_impossible': False, 'question': question, 'answers':[{'text': i[1], 'answer_start': i[2]}]}]}
    trainData.append(tmp)
    idx += 2

os.makedirs('data', exist_ok=True)
with open('data/dialog/distantWithNegativeTrain.json', 'w') as f:
    json.dump(trainData, f)

In [441]:
# model.train_model('data/dialog/distantWithNegativeTrain.json')

In [535]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('bert-base-nli-cls-token')
# model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')
model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

INFO:root:Load pretrained SentenceTransformer: roberta-base-nli-stsb-mean-tokens
INFO:root:Did not find a '/' or '\' in the name. Assume to download model from server.
INFO:root:Downloading sentence transformer model from https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/roberta-base-nli-stsb-mean-tokens.zip and saving it at /Users/jarana/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip
100%|██████████| 459M/459M [02:28<00:00, 3.09MB/s] 
INFO:root:Load SentenceTransformer from folder: /Users/jarana/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip
INFO:root:Use pytorch device: cpu


In [539]:
# Corpus with example sentences
corpus = df_wiki[df_wiki.title == "use uber"].text.tolist()
corpus_embeddings = model.encode(corpus)

Batches: 100%|██████████| 3/3 [00:00<00:00,  3.81it/s]


In [448]:
" ".join(schema.service_name.unique().tolist())

'bank bus calendar event flight home hotel media movie music rent car restaurant taxi service travel weather alarm message payment train'

In [53]:
from nltk.corpus import wordnet as wn
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

In [ ]:
## queries = [camel_case_split(i) for i in list(cand_properties.keys())]
query_embeddings = model.encode(queries)

In [533]:
import scipy
import numpy as np
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist(corpus_embeddings, [query_embedding], "cosine")
    print(query)
    print(corpus[np.argmin(distances)], 1 - distances[np.argmin(distances)][0])
    print()
#     print(distances)
    


taxi stand
taxi 0.8057539461799915

pickup time
pickup 0.8259701335292039

pickup location
pickup 0.8966534259500655

party size
party, 0.7600675358242679

modified time
change 0.5604403547233994

booking time
time 0.5699787148433507

under name
name 0.760251215903371

reservation cancelled
cancel 0.6970501159768268

reservation pending
reservation, 0.7151689708786823

reservation hold
reservation, 0.7946275730114153



In [354]:
intents = []
for idx, row, in schema.iterrows():
    for i in row.intents:
        name = preprocessingText(camel_case_split(i['name']), True)
        desc = preprocessingText(camel_case_split(i['description']), True)
        intent = row.service_name + " " + row.description + " " +name + " " + desc
        intents.append(preprocessingText(intent.lower(), True))

for query in intents:
    tokenized_query = query.split()
    # tokenized_query = tokenizer.tokenize(query)
    doc = bm25.get_top_n(tokenized_query, corpus, n=10)
    print(query)
    print(doc2title[doc[0]])
    text = " ".join(df_wiki[df_wiki.title.str.contains(doc2title[doc[0]])].text.tolist())  
    terms = set(text.split())
    cand_prop = []
    for p in props:
        if p in terms:
            cand_prop.append(p)
    print(cand_prop)
    print()


bank manage bank accounts transfer money check balance check amount money user's bank account
transfer money between bank accounts
['address', 'amount', 'branch', 'code', 'email', 'identifier', 'member', 'name', 'option', 'recipient']

bank manage bank accounts transfer money transfer money transfer money one bank account another user's account
transfer money one bank another
['address', 'amount', 'branch', 'code', 'identifier', 'location', 'name', 'photo', 'procedure', 'recipient', 'review', 'steps']

bus book bus journeys biggest bus network country find bus find bus journey given pair cities
ride bus
['area', 'free', 'map', 'maps', 'steps']

bus book bus journeys biggest bus network country buy bus ticket buy tickets bus journey
drive bus
['course', 'department', 'diagram', 'employees', 'license', 'material', 'position', 'qualifications', 'speed', 'step', 'study']

bus find bus take city want find bus find bus itinerary cities given date
take bus brisbane
[]

bus find bus take city 

/Users/jarana/.local/lib/python3.6/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


[]

event get tickets coolest concerts sports area buy event tickets buy tickets event
get tickets sold event lizardtickets.com
[]

flight find next flight search oneway flight search one-way flights destination
save flight game microsoft flight simulator x
[]

flight find next flight search roundtrip flights search round-trip flights destination
search flights hipmunk
['location', 'text']

flight find next flight reserve oneway flight reserve one-way flight
save flight game microsoft flight simulator x
[]

flight find next flight reserve roundtrip flights reserve round-trip flight
save flight game microsoft flight simulator x
[]

flight search cheap flights across multiple providers search oneway flight search one way flight set preferences
save flight game microsoft flight simulator x
[]

flight search cheap flights across multiple providers search roundtrip flights search round trip flights set preferences
search flights hipmunk
['location', 'text']

home widely used service finding

['event', 'events', 'line', 'option']

event comprehensive portal find reserve seats events near buy event tickets buy tickets event
get tickets sold event lizardtickets.com
[]

flight find one way round trip flights favorite city search oneway flight search one-way flights destination choice
search flights hipmunk
['location', 'text']

flight find one way round trip flights favorite city search roundtrip flights search round-trip flights destination choice
search flights hipmunk
['location', 'text']

home widely used service finding apartments scheduling visits find apartment find apartment city given number bedrooms
make apartment sims 2 apartment life
['area', 'tool']



KeyboardInterrupt: 

In [241]:
props = [camel_case_split(i) for i in orgProp.label.tolist()]

In [15]:
with open("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/tasks.txt") as f:
    for line in f:
        print(line)
        break

{"task_id": "4a06139e", "domain": "UPDATE_CALENDAR", "bot_prompt": "Schedule the user's meeting request", "bot_role": "You are a bot designed to help schedule meetings on a calendar. ", "user_prompt": " You have a meeting saved for March 24th. Ask the chatbot to delete the meeting", "user_role": "You are interacting with a meeting scheduling bot"}



In [330]:
tasks = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/tasks.txt", lines=True)

In [331]:
dialog = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/dialogues/ORDER_PIZZA.txt", lines=True)

In [332]:
import glob

txtfiles = []
for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/dialogues/*.txt"):
    txtfiles.append(file)

In [24]:
metalwoz = pd.read_json(txtfiles[0], lines=True)
for txt in txtfiles[1:]:
    tmp = pd.read_json(txt, lines=True)
    metalwoz = pd.concat([metalwoz,tmp])


In [333]:
slots = []
for idx, row in schema.iterrows():
#     print(row)
    service = row['service_name']
    for i in row.slots:
        slots.append(service+"_"+i['name'])
#     for i in row.intents:
#         slots.extend([service+"_"+j for j in i['required_slots']])
#         slots.extend([service+"_"+j for j in list(i['optional_slots'].keys())])
    

In [335]:
import glob

txtfiles = []
dialog = None
for i in ['train', 'dev', 'test']:
    for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/%s/*.json" % i):
        txtfiles.append(file)
        dialog = pd.concat([dialog, pd.read_json(file)])

        

In [336]:
domains = []
for i in dialog.services.tolist():
    try:
        if len(i) == 1:
            domains.append(i[0])
    except:
        continue

In [337]:
from sklearn.model_selection import train_test_split
trainIds, testIds = train_test_split(list(set(domains)), test_size=0.3, random_state=2020)

In [365]:
prop2desc = {}
schemaT = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/train/schema.json")
schemaT = pd.concat([schemaT, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/dev/schema.json")])
schemaT = pd.concat([schemaT, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/test/schema.json")])
for idx, s in schemaT.iterrows():
    service = s.service_name
    for i in s.slots:
        prop2desc[service+"_"+i['name']] = i['description']


In [367]:
tmp = dialog[dialog.services.str.len() == 1]

In [368]:
tmp['domain'] = [i[0] for i in tmp['services']]

/Users/jarana/anaconda3/envs/torch1/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [428]:
import random
train_data = []
test_data = []

qid = 1
# for idx, row in dialog.iterrows():
# for idx, row in tmp.groupby('domain',as_index = False).apply(lambda x: x.sample(20)).iterrows():
for idx, row in tmp.iterrows():
    frames = row['turns']
    context = []
    
    if str(frames)=='nan':
        continue
        
    for f in frames:
        service = f['frames'][0]['service']
#         service = service.split("_")[0].lower()
        if len(f['frames']) == 1:
            utterance = f['utterance']
            if len(f['frames'][0]['slots']) != 0:
                terms = []
                for s in f['frames'][0]['slots']:
#                   replace text with slot name
#                     _context = " ".join(context)
#                     answer_start = len(_context) + s["start"] + 1
#                     new_answer = utterance[:s["start"]] + s['slot'].replace("_", " ") + utterance[s["exclusive_end"]+1:]
#                     _context += " " + new_answer
#                     _json = {'context': _context, 'qas':[{'id': str(qid), 'is_impossible': False, 
#                                                           'question': prop2desc[service+"_"+s['slot']], 'answers':[{'text':s['slot'].replace("_"," "), 
#                                                            'answer_start': answer_start}]}]}
                
                    _context = " ".join(context)
                    answer_start = len(_context) + s["start"] + 1
#                     new_answer = utterance[:s["start"]] + s['slot'].replace("_", " ") + utterance[s["exclusive_end"]+1:]
                    _context += " " + utterance
                    question = prop2desc[service+"_"+s['slot']]
#                   random negative question
                    neg_question = random.choice(list(prop2desc.values()))
                    while neg_question == question:
                        neg_question = random.choice(list(prop2desc.values()))
                        
                    _json = {'context': _context, 'qas':[{'id': str(qid), 'is_impossible': False, 
                                                          'question': question, 'answers':[{'text':utterance[s['start']: s['exclusive_end']], 
                                                           'answer_start': answer_start}]}, {'id': str(qid+1), 'is_impossible': True, 'question': neg_question, 'answers':[] }]}
    
    
    
    
                    if service in trainIds:
                        train_data.append(_json)
                    else:
                        test_data.append(_json)
                    qid += 2
            context.append(utterance)

In [429]:
len(train_data), len(test_data)

(47399, 25975)

In [430]:
### Save as a JSON file
import json
with open('data/dialog/defaultWithNegativeTrain.json', 'w') as f:
    json.dump(train_data, f)
import json
with open('data/dialog/defaultWithNegativeTest.json', 'w') as f:
    json.dump(test_data, f)

In [432]:
# model.train_model('data/dialog/defaultWithNegativeTrain.json')